# Postwork 6: Automatización y APIs

*Para este postwork utilizamos información que no está relacionada con el dataset que originalmente se está trabajando*

Para realizar peticiones HTTP, se utiliza el paquete requests de Python.

In [1]:
# !pip install requests
import requests

Las peticiones se realizarán a la API de la USGS _(U.S. Geological Survey)_ para obtener información sobre terremotos. La documentación se encuentra en: https://earthquake.usgs.gov/fdsnws/event/1/

El end-point a utilizar es

In [2]:
endpoint = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

Los parámetros que se utilizarán en esta petición son


*   format: Formato de los datos de salida.
*   starttime: Fecha de inicio de incidentes
*   endtime: Fecha de fin de incidentes
*   minlatitude: Latitud mínima en una geometría (región delimitada) rectangular
*   minlongitude: Longitud mínima en una geometría (región delimitada) rectangular
*   maxlatitude: Latitud máxima en una geometría (región delimitada) rectangular
*   maxlongitude: Longitud máxima en una geometría (región delimitada) rectangular

Deseamos obtener los datos de terremotos ocurridos entre el 01/jul/2021 y el 01/ago/2021, en México (aproximadamente entre latitudes 13.607153 a 32.534628 y longitudes -117.124463 a -85.839584) y en formato GeoJSON, el cual es un caso especial de JSON que almacena información geológica.

Por tanto, el objeto de parámetros queda como

In [3]:
payload = {'format': 'geojson',
           'starttime': '2021-07-01',
           'endtime': '2021-08-01',
           'minlatitude': 13.607153,
           'minlongitude': -117.124463,
           'maxlatitude': 32.534628,
           'maxlongitude': -85.839584}

Se realiza la petición GET

In [4]:
r = requests.get(endpoint, params = payload)
r.status_code

200

Se limpiarán los datos a continuación

In [5]:
json = r.json()

Deseamos ver las llaves del objeto

In [6]:
json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

Interesa solo el arreglo **features** pues tiene los datos de los terremotos.

In [7]:
data = json['features']

Este es un ejemplo de los datos que contiene cada elemento en el arreglo

In [8]:
data[0]

{'geometry': {'coordinates': [-104.4992078, 31.60810727, 8.522412109],
  'type': 'Point'},
 'id': 'tx2021oxhx',
 'properties': {'alert': None,
  'cdi': None,
  'code': '2021oxhx',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2021oxhx&format=geojson',
  'dmin': 0.06061087924,
  'felt': None,
  'gap': 74,
  'ids': ',tx2021oxhx,',
  'mag': 1.8,
  'magType': 'ml',
  'mmi': None,
  'net': 'tx',
  'nst': 15,
  'place': '63 km S of Whites City, New Mexico',
  'rms': 0.3,
  'sig': 50,
  'sources': ',tx,',
  'status': 'reviewed',
  'time': 1627774643035,
  'title': 'M 1.8 - 63 km S of Whites City, New Mexico',
  'tsunami': 0,
  'type': 'earthquake',
  'types': ',origin,phase-data,',
  'tz': None,
  'updated': 1628020993172,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2021oxhx'},
 'type': 'Feature'}

Nos interesará cambiar la estructura de este GeoJSON para después pasarlo a un dataframe, además de solo conservar cierta información relevante

In [9]:
earthquakes = []

for feature in data:
  newEarthquakeElem = {
    'id': feature['id'],
    'latitud': feature['geometry']['coordinates'][0],
    'longitud': feature['geometry']['coordinates'][1],
    'url': feature['properties']['url'],
    'lugar': feature['properties']['place'],
    'tipo': feature['properties']['type'],
    'sensacion': feature['properties']['felt'],
    'titulo': feature['properties']['title'],
    'tsunami': feature['properties']['tsunami'],
  }
  earthquakes.append(newEarthquakeElem)

print(earthquakes[0])
print(earthquakes[1])
print(earthquakes[2])

{'id': 'tx2021oxhx', 'latitud': -104.4992078, 'longitud': 31.60810727, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2021oxhx', 'lugar': '63 km S of Whites City, New Mexico', 'tipo': 'earthquake', 'sensacion': None, 'titulo': 'M 1.8 - 63 km S of Whites City, New Mexico', 'tsunami': 0}
{'id': 'tx2021oxhw', 'latitud': -103.5225715, 'longitud': 30.95981644, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2021oxhw', 'lugar': '21 km E of Balmorhea, Texas', 'tipo': 'earthquake', 'sensacion': None, 'titulo': 'M 1.6 - 21 km E of Balmorhea, Texas', 'tsunami': 0}
{'id': 'tx2021oxhj', 'latitud': -104.2584762, 'longitud': 31.60802572, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2021oxhj', 'lugar': '54 km NW of Toyah, Texas', 'tipo': 'earthquake', 'sensacion': None, 'titulo': 'M 2.8 - 54 km NW of Toyah, Texas', 'tsunami': 0}


A continuación, se pasará la información a un dataframe utilizando Pandas

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(earthquakes)

df.head()

,id,latitud,longitud,url,lugar,tipo,sensacion,titulo,tsunami
0,tx2021oxhx,-104.499208,31.608107,https://earthquake.usgs.gov/earthquakes/eventp...,"63 km S of Whites City, New Mexico",earthquake,NaN,"M 1.8 - 63 km S of Whites City, New Mexico",0
1,tx2021oxhw,-103.522571,30.959816,https://earthquake.usgs.gov/earthquakes/eventp...,"21 km E of Balmorhea, Texas",earthquake,NaN,"M 1.6 - 21 km E of Balmorhea, Texas",0
2,tx2021oxhj,-104.258476,31.608026,https://earthquake.usgs.gov/earthquakes/eventp...,"54 km NW of Toyah, Texas",earthquake,NaN,"M 2.8 - 54 km NW of Toyah, Texas",0
3,tx2021oxgn,-103.252434,31.148578,https://earthquake.usgs.gov/earthquakes/eventp...,"20 km WSW of Coyanosa, Texas",earthquake,NaN,"M 3.1 - 20 km WSW of Coyanosa, Texas",0
4,ci39760183,-116.057500,32.404833,https://earthquake.usgs.gov/earthquakes/eventp...,"37km S of Ocotillo, CA",earthquake,NaN,"M 1.5 - 37km S of Ocotillo, CA",0


Y listo. Ahora que tenemos la información en un dataframe, podemos utilizar Pandas para tratar mejor los datos.